# MTH3302 - Méthodes probabilistes et statistiques pour I.A.
#### Polytechnique Montréal
### Projet A2024
----
Équipe T - TODO
### Objectif
Prédire **la consommation en carburant de voitures récentes**.

### Données
Le jeu de données contient pour presque 400 véhicule, la consommation moyenne en L/100km, l'année de frabrication, le type de véhicule, le nombre de cylindre, cylindree, la transmission et la boite.

- `train.cvs` est l'ensemble d'entraînement
- `test.csv` est l'ensemble de test


## Chargement des données

Importation des librairies utilisées dans le calepin.

In [78]:
using CSV, DataFrames, Statistics, Dates, Gadfly, Combinatorics

Premier fichier est l'ensemble des données pour l'entrainement, il contient l'année, le type, le nombre_cylindres, la cylindree, la transmission, la boite, la consommation.

In [79]:
trainData = CSV.read("./data/train.csv", DataFrame)
first(trainData, 1)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite,consommation
,Int64,String31,Int64,String3,String15,String15,String31
1,2023,voiture_moyenne,8,"4,4",integrale,automatique,"13,8358823529412"


Le deuxième fichier est l'ensemble des données pour le test, il contient l'année, le type, le nombre_cylindres, la cylindree, la transmission, la boite. Ici il n'a pas la consommation car il s'agit de la variable d'intérêt.

In [80]:
testData = CSV.read("./data/test.csv", DataFrame)
first(testData, 1)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite
,Int64,String31,Int64,String3,String15,String15
1,2014,voiture_moyenne,4,"2,5",traction,manuelle


## Exploration des données

Lors de l’étape d’exploration des données, notre objectif principal était de mieux comprendre les caractéristiques disponibles afin d’évaluer leur pertinence pour la prédiction de la consommation en carburant. Plutôt que de suivre une méthodologie stricte, nous avons adopté une approche intuitive, en nous appuyant sur nos connaissances de base concernant les véhicules et leur consommation, pour repérer des tendances et des relations potentielles entre les variables.

Comme les colonnes cylindree, et consommation sont inscrites comme String3 et String31 respectivement, nous devons changer leur type dans les deux ensembles de données pour Float64 afin de pouvoir bien les évaluer.

In [81]:
trainData.consommation = parse.(Float64,replace.(trainData.consommation, "," => "."))
trainData.cylindree = parse.(Float64,replace.(trainData.cylindree, "," => "."))
first(trainData, 1)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite,consommation
,Int64,String31,Int64,Float64,String15,String15,Float64
1,2023,voiture_moyenne,8,4.4,integrale,automatique,13.8359


In [82]:
testData.cylindree = parse.(Float64,replace.(testData.cylindree, "," => "."))
first(testData, 1)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite
,Int64,String31,Int64,Float64,String15,String15
1,2014,voiture_moyenne,4,2.5,traction,manuelle
